In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Data Preprocessing

english_sentences = ['start he is a student end', 'start world is temporary end', 'start Many people eat cereal for breakfast end', 'start I am good end', 'start Ted goes to the gym and exercises three times a week end']
russian_sentences = ['start он студент end', 'start мир временен end', 'start Многие люди едят хлопья на завтрак end', 'start У меня все хорошо end', 'start Тед ходит в спортзал и тренируется три раза в неделю. end']

source_tokenizer = Tokenizer()
source_tokenizer.fit_on_texts(english_sentences)
source_sequences = source_tokenizer.texts_to_sequences(english_sentences)
input_texts = pad_sequences(source_sequences, padding='post')

target_tokenizer = Tokenizer()
target_tokenizer.fit_on_texts(russian_sentences)
target_sequences = target_tokenizer.texts_to_sequences(russian_sentences)
target_texts = pad_sequences(target_sequences, padding='post')


In [2]:
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Concatenate
from tensorflow.keras.layers import AdditiveAttention as Attention
from tensorflow.keras.models import Model

# Model parameters
embedding_dim = 256
latent_dim = 512
num_encoder_tokens = len(source_tokenizer.word_index) + 1
num_decoder_tokens = len(target_tokenizer.word_index) + 1

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
# Attention Layer
attention = Attention()
attention_output = attention([decoder_outputs, encoder_outputs])

# Concatenating attention output and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_output])

# Dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

In [3]:
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
from tensorflow.keras.utils import to_categorical
decoder_target_data = to_categorical(target_texts, num_decoder_tokens)
model.fit([input_texts, target_texts], decoder_target_data, batch_size=64, epochs=50, validation_split=0.2)


Epoch 1/50
1/1 [==============================] - 7s 7s/step - loss: 3.2499 - accuracy: 0.0208 - val_loss: 3.2603 - val_accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 1s 574ms/step - loss: 2.9783 - accuracy: 0.5417 - val_loss: 3.2678 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 451ms/step - loss: 2.5398 - accuracy: 0.5417 - val_loss: 3.4240 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 478ms/step - loss: 2.0151 - accuracy: 0.5417 - val_loss: 3.2771 - val_accuracy: 0.0000e+00
Epoch 5/50
1/1 [==============================] - 0s 456ms/step - loss: 2.4741 - accuracy: 0.5833 - val_loss: 3.3109 - val_accuracy: 0.0000e+00
Epoch 6/50
1/1 [==============================] - 0s 465ms/step - loss: 1.8097 - accuracy: 0.5417 - val_loss: 3.7752 - val_accuracy: 0.0000e+00
Epoch 7/50
1/1 [==============================] - 0s 492ms/step - loss: 1.6881 - accuracy: 0.5417 - val_loss: 3.3319 - val_accuracy: 0.0833

In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            7168      ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            6656      ['input_2[0][0]']             
                                                                                              